In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ising_model
import exact_ising_model as exact
import plotting
import analysis
import simulation

# Metropolis
Lattice Size 4, Temperature 8.0
Energy per Site: -0.2610626220703125 +/- 0.0018771324458461633
Magnetization per Site: -1.9073486328125e-06 +/- 0.002077072882542814
m (<|M|>/N): 0.26807212829589844 +/- 0.0010193575963454566
Energy Squared: 54.25830078125 +/- 0.44047312300298547
Magnetization Squared: 28.64300537109375 +/- 0.18814095808833015
Heat Capacity per Site: 0.03599535199100501 +/- 0.00038559787097370484

# Wolff
Lattice Size 4, Temperature 8.0
Energy per Site: -0.267791748046875 +/- 0.0032271910499994372
Magnetization per Site: -0.0035552978515625 +/- 0.00286548199467902
m (<|M|>/N): 0.2700157165527344 +/- 0.0012436922244417024
Energy Squared: 55.67626953125 +/- 0.6168333494425499
Magnetization Squared: 28.962158203125 +/- 0.23760101756071275
Heat Capacity per Site: 0.036492407917194214 +/- 0.0004084473137390334

# Corboz
Energy per site : -0.263511657715  +/-  0.00150418036608
Magnetization per site : 0.000926971435547  +/-  0.00199863921378
m : 0.268798828125  +/-  0.000809964714865
Energy squared : 54.658203125  +/-  0.391778044629
Magnetization squared : 28.7371826172  +/-  0.149540865285

In [ ]:
simulation.simulation_range("ising", "wolff", [10, 20, 40], 1, "lo", 5000, 65536, 2.169, 2.369, step=0.01)

In [ ]:
# T=2
10 by 10: 1 loop, best of 1: 33.7 s per loop
20 by 20: 1 loop, best of 1: 1min 20s per loop
        
# T=3
10 by 10: 1 loop, best of 1: 11.6 s per loop
20 by 20: 1 loop, best of 1: 10.8 s per loop

In [ ]:
files = ["20160510_112747_ising_wolff_10_65536_[2-3]_0.1.pickle",
"20160510_113429_ising_wolff_20_65536_[2-3]_0.1.pickle",
"20160510_115100_ising_wolff_40_65536_[2-3]_0.1.pickle"]
files = ["20160510_112747_ising_wolff_10_65536_[2-3]_0.1.pickle", 
        "20160510_113429_ising_wolff_20_65536_[2-3]_0.1.pickle",
        "20160510_115100_ising_wolff_40_65536_[2-3]_0.1.pickle"]
(critical_temperature,
 critical_temperature_error,
 magnetizabilities,
 magnetizations,
 heat_capacities) = analysis.data_analysis(files)

In [ ]:
analysis.find_critical_exponents(critical_temperature,
                                 critical_temperature_error,
                                 magnetizabilities,
                                 magnetizations,
                                 heat_capacities, alpha=0.4, beta=1/8, gamma=7/4, nu=1)

In [ ]:
analysis.chi_squared_data_collapse(magnetizabilities, "Magnetizability", 2.269, critical_temperature_error)

In [ ]:
data = analysis.open_simulation_files(["20160513_150716_ising_wolff_10_65536_[2.269-2.269]_0.1.pickle",
"20160513_150831_ising_wolff_20_65536_[2.269-2.269]_0.1.pickle",
"20160513_151121_ising_wolff_40_65536_[2.269-2.269]_0.1.pickle",
"20160513_151640_ising_wolff_60_65536_[2.269-2.269]_0.1.pickle",
"20160513_152459_ising_wolff_80_65536_[2.269-2.269]_0.1.pickle"])

In [ ]:
lattice_sizes = []
magnetizations_log = []

import scipy.stats

for k in data:
    lattice_size = np.log(k[0][0][0])
    magnetization_log = np.log(k[0][1]["m"])
    lattice_sizes.append(lattice_size)
    magnetizations_log.append(magnetization_log)
    print(lattice_size, magnetization)
    
lattice_sizes = np.asarray(lattice_sizes)
magnetizations_log = np.asarray(magnetizations_log)
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(lattice_sizes, magnetizations_log)
print(-slope, std_err)
plt.plot(lattice_sizes, magnetizations_log)
plt.show()



In [ ]:
import itertools

def chi_squared_data_collapse(data, quantity, critical_temperature,
                              critical_temperature_error, ratio, ratio_error):
    """Find the critical exponents for the magnetizability."""
    beta_estimate = []
    nu_estimate = []
    for k in [-1, 0, 1]:
        critical_temperature += k * critical_temperature_error
        for p in [-1, 0, 1]:
            lowest_residual = np.inf
            best_nu = 0
            best_beta = 0
            for nu in np.linspace(0, 1, 101):
                beta = (ratio + p * ratio_error) * nu
                scaling_functions = []
                for d in data:
                    scaling_function_at_size = []
                    lattice_size = d[0]
                    values = d[1]
                    for v in values:
                        t = (v[0] - critical_temperature) / critical_temperature
                        scaling_variable = lattice_size**(1 / nu) * t
                        v_tilde = lattice_size**(beta / nu) * v[1]
                        # print(v_tilde, lattice_size**(gamma / nu), v[1])
                        scaling_function_at_size.append((scaling_variable, v_tilde))
                    scaling_functions.append([lattice_size, scaling_function_at_size])

                # print("alpha={0}, nu={1}".format(alpha, nu))
                # print(scaling_functions)
                at_interval = [a for a in list(itertools.chain(*[s[1] for s in scaling_functions])) if -5 <= a[0] <= 2.5]
                if len(at_interval) >= 10:
                    at_interval_x, at_interval_y = zip(*at_interval)
                    polynomial, residuals, _, _, _ = np.polyfit(at_interval_x, at_interval_y, 5, full=True)
                    if residuals < lowest_residual:
                        lowest_residual = residuals
                        best_nu = nu
                        best_beta = beta
#                         print("beta={0}, nu={1}, residual={2}".format(best_beta, best_nu, residuals))
#                         f = np.poly1d(polynomial)

#                         x_new = np.linspace(min(at_interval_x), max(at_interval_x), 50)
#                         y_new = f(x_new)

#                         plt.plot(at_interval_x, at_interval_y, 'o', x_new, y_new)
#                         plt.show()
            print("best beta = {0}, best nu = {1}".format(best_beta, best_nu))
            beta_estimate.append(best_beta)
            nu_estimate.append(best_nu)
            
    print(np.mean(beta_estimate), analysis.calculate_error(beta_estimate))
    print(np.mean(nu_estimate), analysis.calculate_error(nu_estimate))

In [ ]:
chi_squared_data_collapse(magnetizations, "Magnetization", critical_temperature, critical_temperature_error, -slope, std_err)

In [ ]:
files = ["20160513_153432_ising_wolff_10_65536_[2.169-2.369]_0.01.pickle",
        "20160513_155413_ising_wolff_20_65536_[2.169-2.369]_0.01.pickle",
        "20160513_164844_ising_wolff_40_65536_[2.169-2.369]_0.01.pickle"]
(critical_temperature,
 critical_temperature_error,
 magnetizabilities,
 magnetizations,
 heat_capacities) = analysis.data_analysis(files)

In [ ]:
analysis.data_collapse(magnetizabilities, "Magnetizability", critical_temperature, 0.2, 1, "beta", "nu")

In [ ]:
analysis.chi_squared_data_collapse(heat_capacities, critical_temperature, critical_temperature_error)

In [ ]:
analysis.data_collapse(magnetizabilities, "Magnetizability", 2.269, 0.125, 1, "beta", "nu")

In [ ]:
ising = ising_model.IsingModel(10, 1, 2, "hi", 2000)
%lprun -f ising.wolff ising.wolff()